# Setup

The following needs to be done at the start of every session.

In [1]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

In [4]:
import julia
# julia.install()

In [5]:
from julia import Bigsimr as bs
from julia import Distributions as dist
from julia.Bigsimr import Pearson, Spearman, Kendall
from julia.Bigsimr import rvec, cor, cor_randPD, cor_nearPD, cor_convert, cor_bounds
from julia.Bigsimr import pearson_match, pearson_bounds

In [6]:
import numpy as np
import pandas as pd
from pydataset import data

# Getting Started

We’re going to show the basic use and syntax of *bigsimr* by using the New York air quality data set (airquality) included in the *RDatasets* package. We will focus specifically on the temperature (degrees Fahrenheit) and ozone level (parts per billion).

In [7]:
df = data("airquality") # requires R installation
df = df[["Ozone", "Temp"]].dropna()
df.head(10)

,Ozone,Temp
1,41.0,67
2,36.0,72
3,12.0,74
4,18.0,62
6,28.0,66
7,23.0,65
8,19.0,59
9,8.0,61
11,7.0,74
12,16.0,69


In [8]:
df.shape

(116, 2)

We can see that not all margins are normally distributed; the ozone level is highly skewed. Though we don’t know the true distribution of ozone levels, we can go forward assuming that it is log-normally distributed.

To simulate observations from this joint distribution, we need to estimate the correlation and the marginal parameters.

## Estimating Correlation

To estimate the correlation, we use `cor` with an argument specifying the type of correlation to estimate. The options are `Pearson`, `Spearman`, or `Kendall`.

In [9]:
p = cor(np.asmatrix(df))
p

array([[1.        , 0.69836034],
       [0.69836034, 1.        ]])

## Defining Marginal Distributions

Next we can estimate the marginal parameters. Assuming that the `Temperature` is normally distributed, it has parameters:

In [10]:
mu_temp = np.mean(df.Temp)
sd_temp = np.std(df.Temp)
(mu_temp, sd_temp)

(77.87068965517241, 9.444511426013516)

and assuming that `Ozone` is log-normally distributed, it has parameters:

In [11]:
mu_ozone = np.mean(np.log(df.Ozone))
sd_ozone = np.sqrt(np.mean((np.log(df.Ozone) - np.mean(np.log(df.Ozone)))**2))
(mu_ozone, sd_ozone)

(3.4185151008120065, 0.8617359690270705)

Finally we take the parameters and put them into a vector of margins:

In [12]:
margins = [
    dist.Normal(mu_temp, sd_temp),
    dist.LogNormal(mu_ozone, sd_ozone)
]
margins

[<PyCall.jlwrap Distributions.Normal{Float64}(μ=77.87068965517241, σ=9.444511426013516)>,
 <PyCall.jlwrap Distributions.LogNormal{Float64}(μ=3.4185151008120065, σ=0.8617359690270705)>]

## Multivariate Distribution

While the individual components can be used separately within the package, they work best when joined together into a `MvDistribution` data type:

In [13]:
D = bs.MvDistribution(p, margins, Pearson)
D

<PyCall.jlwrap Bigsimr.MvDistribution([1.0 0.6983603421509319; 0.6983603421509319 1.0], Distributions.Distribution{Distributions.Univariate,Distributions.Continuous}[Distributions.Normal{Float64}(μ=77.87068965517241, σ=9.444511426013516), Distributions.LogNormal{Float64}(μ=3.4185151008120065, σ=0.8617359690270705)], Bigsimr.Pearson)>

## Correlation Bounds

Given a vector of margins, the theoretical lower and upper correlation coefficients can be estimated using simulation:

In [14]:
bounds = cor_bounds(D)
bounds.lower, bounds.upper

(array([[ 1.        , -0.82670376],
        [-0.82670376,  1.        ]]),
 array([[1.        , 0.82627587],
        [0.82627587, 1.        ]]))

The `pearson_bounds` function uses more sophisticated methods to determine the theoretical lower and upper Pearson correlation bounds. It also requires more computational time.

In [15]:
bounds = pearson_bounds(D)
bounds.lower, bounds.upper

(array([[ 1.        , -0.82112233],
        [-0.82112233,  1.        ]]),
 array([[1.        , 0.82112233],
        [0.82112233, 1.        ]]))

## Simulating Data

Let’s now simulate 10,000 observations from the joint distribution using `rvec`:

In [16]:
x = rvec(10_000, p, margins)
x

array([[ 76.49207456,  24.94288087],
       [ 94.41639329, 226.64958757],
       [ 80.61791841,  41.70336123],
       ...,
       [ 87.8246877 ,  33.07118316],
       [ 89.53425449,  64.62870939],
       [ 93.39659825, 308.04864777]])

Alternatively we can use `rand` with the `MvDistribution` type:

In [17]:
bs.rand(D, 10)

array([[78.16351388, 20.61558617],
       [61.42101374, 10.21764271],
       [84.55249093, 84.56997442],
       [86.27983984, 35.15753168],
       [82.63533131, 36.0237821 ],
       [95.13233123, 57.71481619],
       [68.49710554,  3.58408481],
       [73.88646591, 52.44551657],
       [76.91876108, 38.24262044],
       [81.72553032, 52.09668687]])

# Pearson Matching

## Correlation Conversion

Let's say we really wanted to estimate the Spearman correlation between the temperature and ozone.

In [18]:
ps = cor(np.asmatrix(df), Spearman)
ps

array([[1.        , 0.77404296],
       [0.77404296, 1.        ]])

In [19]:
cor_bounds(margins[0], margins[1], Spearman)

<PyCall.jlwrap (lower = -1.0, upper = 1.0)>

If we just use the Spearman correlation when we simulate data, then the errors are double.

1. The NORTA algorithm is expecting a Pearson correlation
2. The non-linear transformation in the NORTA step does not guarantee that the input correlation is the same as the output.

In [20]:
D2 = bs.MvDistribution(ps, margins, Spearman)
x2 = bs.rand(D2, 1_000_000)

In [21]:
cor(x2, Spearman)

array([[1.        , 0.77465057],
       [0.77465057, 1.        ]])

Let's try to address **(1)** and convert the Spearman correlation to a Pearson correlation.

In [22]:
pp = cor_convert(ps, Spearman, Pearson)
D3 = bs.MvDistribution(pp, margins, Pearson)
x3 = bs.rand(D3, 1_000_000)

In [23]:
cor(x3, Pearson)

array([[1.        , 0.64828674],
       [0.64828674, 1.        ]])

In [24]:
cor(x3, Spearman)

array([[1.        , 0.77411538],
       [0.77411538, 1.        ]])

Notice that the estimated Pearson correlation is lower than the target Pearson correlation, but the estimated Spearman correlation is essentially the same as the target. This is because the transformation in the NORTA step is monotonic, which means that rank-based correlations are preserved. As a consequence, we can match the Spearman correlation exactly (up to stochastic error), but not the Pearson.

## Pearson Matching

We can overcome this limitation by employing a Pearson matching algorithm that determines the necessary input correlation in order to achieve the target correlation. Let's now address **(2)**.

In [25]:
D4 = pearson_match(D2)
cor(D4)

array([[1.        , 0.94266461],
       [0.94266461, 1.        ]])

Notice the significant change in the input correlation!

In [26]:
x4 = bs.rand(D4, 1_000_000)
cor(x4, Pearson)

array([[1.        , 0.77534013],
       [0.77534013, 1.        ]])

But the estimated correlation is nearly spot on to the [converted] Pearson correlation (ρ_p).

A better example is using the `MvDistribution`. We never estimated the correlation after simulating, so let's look at that now.

In [27]:
cor(bs.rand(D, 1_000_000))

array([[1.        , 0.57212337],
       [0.57212337, 1.        ]])

compared to the target correlation:

In [28]:
p

array([[1.        , 0.69836034],
       [0.69836034, 1.        ]])

The estimated correlation is much too low. Let's do some Pearson matching and observe the results.

In [29]:
D5 = pearson_match(D)
x5 = bs.rand(D5, 1_000_000)
cor(x5)

array([[1.        , 0.69859194],
       [0.69859194, 1.        ]])

Now the simulated data results in a correlation structure that exactly matches the target Pearson correlation!

# Nearest Correlation Matrix

Sometimes what we want really is the Spearman correlation. Then we don't need to do any Pearson matching. All we need to do is estimate/obtain the Spearman correlation of some data, convert it to Pearson, and then simulate. The resulting simulated data will have the same Spearman correlation as the one estimated from the data (up to stochastic error). The problem is that for high dimensional data, the Spearman or converted Pearson correlation matrix may not be positive semidefinite (PSD). The problem is then how to compute the nearest PSD correlation matrix.

We provide the function `cor_nearPD` to handle this problem. It is based off of the work of Qi and Sun (2006), and is a quadratically convergent algorithm. Here we use a synthetic non-positive definite correlation matrix.

In [30]:
s = cor_randPD(200)
np.linalg.cholesky(s) # succeeds: is positive definite

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.57504413e-02,  9.97126808e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.82804420e-02,  8.73421400e-03,  9.96057366e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.20443614e-02, -3.07842939e-02, -2.52114027e-03, ...,
         2.44499208e-01,  0.00000000e+00,  0.00000000e+00],
       [-5.02600673e-02, -8.10271306e-02, -4.38855116e-04, ...,
         1.74158693e-02,  2.53672536e-01,  0.00000000e+00],
       [ 1.41010863e-01, -5.01500642e-02,  1.01045126e-01, ...,
        -4.35599558e-02,  8.39738200e-02,  2.21202353e-01]])

In [31]:
r = cor_convert(s, Spearman, Pearson)
np.linalg.cholesky(r) # fails: not positve definite

LinAlgError: Matrix is not positive definite

We see that the converted Pearson correlation matrix is no longer positive definite. This will result in a failure during the multivariate normal generation, particularly during the Cholesky decomposition. We can fix this by computing the nearest PD correlation.

In [32]:
p = cor_nearPD(r)
np.linalg.cholesky(p) # succeeds: is positive definite

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.90429146e-02,  9.96871214e-01,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.17751621e-02,  7.65194346e-03,  9.95750354e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.58704528e-02, -3.26870947e-02, -4.86666076e-03, ...,
         3.05921932e-03,  0.00000000e+00,  0.00000000e+00],
       [-5.23622595e-02, -8.52199480e-02, -1.73378985e-03, ...,
        -8.52436881e-04,  3.45355229e-03,  0.00000000e+00],
       [ 1.47351550e-01, -5.16426504e-02,  1.05840022e-01, ...,
        -7.89503807e-04,  5.53183837e-04,  3.08041080e-03]])